Cell 1: Activation Cell

In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

from src.train import train_model

DATA_DIR = PROJECT_ROOT / "data" / "raw" / "EuroSAT_RGB"

model, class_names = train_model(
    data_dir=DATA_DIR,
    epochs=5
)

Using device: cpu


Epoch 1/5: 100%|██████████| 296/296 [00:40<00:00,  7.25it/s]


Epoch 1 | Train Loss: 0.9991 | Val Acc: 0.7454


Epoch 2/5: 100%|██████████| 296/296 [00:40<00:00,  7.28it/s]


Epoch 2 | Train Loss: 0.5644 | Val Acc: 0.7960


...

Epoch 3 | Train Loss: 0.4241 | Val Acc: 0.8281


Epoch 4/5: 100%|██████████| 296/296 [00:58<00:00,  5.06it/s]


Epoch 4 | Train Loss: 0.3176 | Val Acc: 0.8247


Epoch 5/5: 100%|██████████| 296/296 [01:03<00:00,  4.64it/s]


Epoch 5 | Train Loss: 0.2206 | Val Acc: 0.8395
Model saved to models\simple_cnn.pth
